In [17]:
from sklearn.neighbors import KNeighborsClassifier
import warnings
import math
import pandas as pd
import numpy as np
import random
from deap import creator, base, tools, algorithms
from sklearn import svm
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings('ignore')

In [18]:
def avg(l):
    """
    Returns the average between list elements
    """
    return (sum(l)/float(len(l)))

In [19]:
def calculate_equivalence_classes(data, condition_attrs):
    # 取出条件属性的列索引
    # 设置条件属性为索引
    # print(condition_attrs)
    data_indexed = data.set_index(condition_attrs)
    # 按照索引分组并将分组内结果合并为列表
    equivalence_classes = data_indexed.groupby(level=condition_attrs)[data.columns[0]].apply(list).to_dict()
    return equivalence_classes

def getFitness(data,decision_attr,individual):
    # 计算D的信息熵
    total_records = len(data)
    decision_counts = data[decision_attr].value_counts()
    decision_prob = decision_counts / total_records
    entropy_D = -(decision_prob * np.log2(decision_prob)).sum()

    feature_columns = [col for col in data.columns if col not in ['id','target']]
    selected_features = [feature_columns[i] for i in range(len(feature_columns)) if individual[i] == 1]

    equivalence_classes = calculate_equivalence_classes(data,selected_features)
    equivalence_classes_dec = calculate_equivalence_classes(data, decision_attr)

    rule_confidences = []
    entropy_DB = 0
    total_rule=0

    for equivalence_class in equivalence_classes.values():
        class_size = len(equivalence_class)
        class_prob = class_size / total_records
        # print('-----------')
        # print(equivalence_class)
        for key_decision in equivalence_classes_dec.keys():
            # print(key_decision)
            x=set(equivalence_class).intersection(equivalence_classes_dec.get(key_decision))
            # print(x)
            if(len(x)>total_records*0.01):
                total_rule+=len(x)
                y = 1-len(x) / class_size
                rule_confidences.append(y)
            if (len(x)>0):
                decision_prob = len(x) / class_size
                entropy_DB -= class_prob * decision_prob * math.log(decision_prob, 2)

    rule_confidences = np.array(rule_confidences)
    # print(rule_confidences)
    count = len(rule_confidences)
    if count == 0:
        return(999,);
    if total_rule/total_records<0.3:
        return (999,);

    avg_confidence = np.sum(rule_confidences) / count +wight1*(entropy_D-entropy_DB)+wight2*(count/total_records)
    return(avg_confidence,)

def geneticAlgorithm(df,y,n_population, n_generation):
    """
    Deap global variables
    Initialize variables to use eaSimple
    """
   # random.seed(42)
    # create individual
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    # create toolbox
    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat,
                     creator.Individual, toolbox.attr_bool, len(X.columns))
    toolbox.register("population", tools.initRepeat, list,
                     toolbox.individual)
    toolbox.register("evaluate", getFitness,df,y)
    toolbox.register("mate", tools.cxOnePoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)

    # initialize parameters
    pop = toolbox.population(n=n_population)
    hof = tools.HallOfFame(n_population * n_generation)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    # stats.register("max", np.max)

    # genetic algorithm
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2,
                                   ngen=n_generation, stats=stats, halloffame=hof,
                                   verbose=True)

    # return hall of fame
    top_5_hof = sorted(hof.items, key=lambda x: x.fitness.values[0])[:1]
    for i, ind in enumerate(top_5_hof):
        print(f"Top {i+1} solution: {ind}\nFitness: {ind.fitness.values[0]}\n")
    return top_5_hof
    # return hof

In [20]:
def bestIndividual_KNN(hof,df):
    k = 5
    knn = KNeighborsClassifier(n_neighbors=k)
    for ind in hof:
        df = pd.read_csv('data/cre/crx.csv')
        feature_columns = [col for col in df.columns if col not in ['id','target']]
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]
        Y=df["target"]
        X=df[selected_features]
        kf = KFold(n_splits=10, shuffle=True, random_state=42)
        accuracy_scores = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

            knn.fit(X_train, y_train)
            y_pred = knn.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            accuracy_scores.append(accuracy)
        mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
        return mean_accuracy,selected_features

In [21]:
from sklearn.model_selection import StratifiedKFold
def bestIndividual_SVM(hof, df):
    def SVM(trainX, trainY, testX, testY):
        scaler = StandardScaler()
        scaler.fit(trainX)
        trainX3 = scaler.transform(trainX)
        testX3 = scaler.transform(testX)
        clf = svm.SVC(C=10, gamma=0.1, max_iter=500)
        clf.fit(trainX3, trainY)
        test_result = clf.predict(testX3)
        precision = np.sum(test_result == testY) / testY.shape[0]
        return precision

    maxaccuracy = 0.0
    for ind in hof:
        df = pd.read_csv('data/cre/crx.csv')
        feature_columns = [col for col in df.columns if col not in ['id', 'target']]
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]
        Y = df["target"]
        X = df[selected_features]

        # Use stratified k-fold cross-validation
        skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        scores = []
        for train_index, test_index in skf.split(X, Y):
            trainX, testX = X.iloc[train_index], X.iloc[test_index]
            trainY, testY = Y.iloc[train_index], Y.iloc[test_index]
            score = SVM(trainX, trainY, testX, testY)
            scores.append(score)
        avg_score = np.mean(scores)

        if avg_score > maxaccuracy:
            maxaccuracy = avg_score
            best_feature = selected_features

    return maxaccuracy, best_feature

In [22]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

def bestIndividual_DT(hof, df):
    for ind in hof:
        df = pd.read_csv('data/cre/crx.csv')
        feature_columns = [col for col in df.columns if col not in ['id','target']]
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]
        X = df[selected_features]
        Y = df['target']
        dt = DecisionTreeClassifier()
        kf = KFold(n_splits=10, shuffle=True, random_state=42)

        accuracy_scores = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

            dt.fit(X_train, y_train)

            y_pred = dt.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            accuracy_scores.append(accuracy)

        mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
    return mean_accuracy

In [23]:
wighta=[0,0.1,0.2,0.3,0.4,0.5]
wightb=[0,1,2,3,4,5]
SVM=[[0 for _ in range(len(wighta))] for _ in range(len(wighta))]
KNN=[[0 for _ in range(len(wighta))] for _ in range(len(wighta))]
DT=[[0 for _ in range(len(wighta))] for _ in range(len(wighta))]
F=[[0 for _ in range(len(wighta))] for _ in range(len(wighta))]


for i_index, i in enumerate(wighta):
    for j_index, j in enumerate(wightb):
        try:
            # 你的代码
            wight1=i
            wight2=j
            n_pop=30
            n_gen=30
            # read dataframe from csv
            df = pd.read_csv('data/cre/crx.csv')
            # encode labels column to numbers
            y = df['target']
            feature_columns = ['f3','f8','f14','f15']
            for col in feature_columns:
                df[col] = pd.qcut(df[col], q=5, labels=False, duplicates='drop')
            feature_columns = [ col for col in df.columns if col not in ['id','target']]
            X = df[feature_columns]
            # for col in feature_columns:
            #     df[col] = pd.qcut(df[col], q=5, labels=False, duplicates='drop')
            # get accuracy with all features
            individual = [1 for i in range(len(X.columns))]
            getFitness(df,'target',individual)
            print("Accuracy with all features: \t" +
                  str(getFitness(df,'target',individual)) + "\n")

            # apply genetic algorithm
            hof = geneticAlgorithm(df,'target',n_pop, n_gen)

            print('------------wight1--------------')
            print(wight1)
            print('------------wight2--------------')
            print(wight2)

            accuracy_SVM,individual = bestIndividual_SVM(hof,df)
            SVM[i_index][j_index] = accuracy_SVM
            print("-----------SVM---------------")
            print(accuracy_SVM, "保存至 SVM[{}][{}]".format(i_index, j_index))

            accuracy_KNN,individual = bestIndividual_KNN(hof,df)
            KNN[i_index][j_index] = accuracy_KNN
            print("-----------KNN---------------")
            print(accuracy_KNN, "保存至 KNN[{}][{}]".format(i_index, j_index))

            accuracy_DT = bestIndividual_DT(hof,df)
            DT[i_index][j_index] = accuracy_DT
            print("-----------DT---------------")
            print(accuracy_DT, "保存至 DT[{}][{}]".format(i_index, j_index))

            print('------------特征-----------------')
            print(individual)
            F[i_index][j_index] = individual
            print("特征保存至 F[{}][{}]".format(i_index, j_index))

        except Exception as e:
            print("执行过程出现错误：", e)
            continue

Accuracy with all features: 	(999,)

gen	nevals	avg    	min    
0  	30    	965.705	0.14896
1  	19    	865.818	0.0832965
2  	24    	599.465	0.0486538
3  	17    	366.411	0.0619883
4  	16    	66.7593	0.0619883
5  	21    	33.4217	0.0619883
6  	19    	0.0846043	0.0619883
7  	16    	33.3614  	0.0619883
8  	13    	33.3648  	0.0619883
9  	21    	33.3647  	0.0619883
10 	19    	66.6579  	0.0619883
11 	24    	66.6622  	0.0619883
12 	17    	133.255  	0.0619883
13 	14    	33.3645  	0.0619883
14 	21    	0.0612866	0.0379272
15 	12    	0.0636983	0.0379272
16 	21    	0.0652112	0.0379272
17 	14    	66.657   	0.0379272
18 	20    	33.356   	0.0379272
19 	18    	133.252  	0.0379272
20 	19    	66.6376  	0.0379272
21 	25    	66.639   	0.0379272
22 	18    	0.0436318	0.0379272
23 	25    	33.3367  	0.0379272
24 	12    	0.0435586	0.0379272
25 	18    	66.6407  	0.0379272
26 	20    	33.3432  	0.0379272
27 	20    	99.9391  	0.0379272
28 	16    	33.3418  	0.0379272
29 	17    	66.6402  	0.0379272
30 	20    	99.9432  

In [24]:
print(SVM)
print(KNN)
print(DT)
print(F)


[[0.855072463768116, 0.8608695652173912, 0.8521739130434783, 0.8623188405797102, 0.8710144927536232, 0.8507246376811596], [0.855072463768116, 0.8710144927536232, 0.8753623188405797, 0.8681159420289856, 0.8637681159420291, 0.8594202898550725], [0.8753623188405797, 0.846376811594203, 0.8507246376811594, 0.8594202898550725, 0.8594202898550725, 0.8478260869565217], [0.8492753623188406, 0.8608695652173914, 0.8623188405797102, 0.8318840579710145, 0.844927536231884, 0.8579710144927537], [0.8681159420289856, 0.8434782608695652, 0.855072463768116, 0.8623188405797102, 0.8608695652173914, 0.8478260869565218], [0.8623188405797102, 0.8376811594202899, 0.7565217391304349, 0.8594202898550725, 0.7318840579710144, 0.7304347826086957]]
[[0.7130434782608696, 0.7376811594202899, 0.6057971014492753, 0.6333333333333334, 0.6478260869565218, 0.6811594202898551], [0.7130434782608696, 0.7318840579710146, 0.7869565217391304, 0.7652173913043476, 0.6318840579710145, 0.7478260869565218], [0.8057971014492754, 0.7333